In [1]:
import sys, os
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
from matplotlib.backends.backend_pdf import PdfPages


from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)


%matplotlib inline

%load_ext autoreload
%autoreload 2

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [3]:
def valid_hp(cat, nside = 4096):
    
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=nside, nest= False)
    hpind_reject = hpRaDecToHEALPixel(cat['RA'][cat['VETO_FLAG'] == 0], 
                                    cat['DEC'][cat['VETO_FLAG'] == 0], nside=nside, nest= False)
    
    hppixnumber = np.arange(hp.nside2npix(nside))
    
    geomask = np.in1d( hppixnumber, hpind )
    rejectmask = np.in1d( hppixnumber, hpind_reject )
    
    print(np.sum(geomask), np.sum(~rejectmask))
    
    
    totalmask = geomask * (~rejectmask)
    return hppixnumber[totalmask]


def hp_mask(cat, valid_hpind, nside = 4096):
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=nside, nest= False)
    HPmask = np.in1d(hpind, valid_hpind)
    return HPmask

def callingBOSSmask(option='SGC'):
    
    if option == 'SGC':
        valid_hpind = fitsio.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits')
    elif option == 'NGC': 
        valid_hpind = fitsio.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits')

    CleanMask = np.zeros((valid_hpind.size, ), dtype=[('PIXEL', 'i4'),('RA', 'f8'), ('DEC', 'f8')])
    CleanMask['PIXEL'] = valid_hpind
    sys_ra, sys_dec = hpHEALPixelToRaDec(valid_hpind, nside = 4096)
    CleanMask['RA'] = sys_ra
    CleanMask['DEC'] = sys_dec
    return CleanMask


def BOSSvetoing(cat, option='SGC'):
    
    if option == 'SGC':
        valid_hpind = esutil.io.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits')
    elif option == 'NGC': 
        valid_hpind = esutil.io.read('/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits')
    elif option =='all':
        valid_hpind = esutil.io.read(['/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12NGC-COMPSAM_V6C.fits',
                                   '/n/des/lee.5922/data/cmass_cat/vetomask/mask_nside4096_Patchy-Mocks-Randoms-DR12SGC-COMPSAM_V6C.fits'],
                                 combine=True)
    hpind = hpRaDecToHEALPixel(cat['RA'], cat['DEC'], nside=4096, nest= False)
    HPmask = np.in1d(hpind, valid_hpind)
    
    return cat[HPmask]

In [4]:
def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/fs/scratch/PCON0003/warner785/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

In [7]:
## :means I will add these back in later when I have them

rootdir = '../output/test/train_cat/y3/'
##dmass_spt = io.SearchAndCallFits(path = rootdir, columns = None, keyword = 'dmass_spt_sys_v3.fits')
##dmass_spt = dmass_spt[ dmass_spt['CMASS_PROB'] > 0.01 ]
dmass_st82 = io.SearchAndCallFits(path = rootdir, columns = None, keyword = 'dmass_st82_DET200_final.fits')
##dmass_st82 =dmass_st82[ dmass_st82['EachProb_CMASS'] > 0.01]

import fitsio
cmass = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
train_sample = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
#train_sample = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')   
train = keepGoodRegion(train_sample)

#fitsio.write( output_dir+'/cmass_in_st82.fits', train_sample)

print('num of train_sample after des veto', train.size)

##dmass_spt = Cuts.keepGoodRegion(dmass_spt)
dmass_st82 = keepGoodRegion(dmass_st82)

print(np.sum(dmass_st82['CMASS_PROB']))
##print np.sum(dmass_spt['CMASS_PROB'])
print(train.size)


Calling catalogs >>>
dmass_st82_DET200_final.fits
reading 1/1 ../output/test/train_cat/y3/dmass_st82_DET200_final.fits
total num of train 258884

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 61031
28502.419064939113
61031


In [8]:
# calling map 
path = '/fs/scratch/PCON0008/warner785/bwarner/'
LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits') #BAO is different measurement, may be different from this analysis
# 'Y1LSSmask_v2_redlimcut_il22_seeil4.0_4096ring.fits'
#GoldMask = callingEliGoldMask()
LSSMask = LSSGoldmask
GoldMask = LSSGoldmask

pixra, pixdec = hp.pix2ang(nside=4096,ipix=GoldMask['PIXEL'],nest=True,lonlat=True)

LSSMask = LSSMask[pixdec >-3.0 ]
#GoldMask_st82 = Cuts.SpatialCuts(GoldMask, ra=320, ra2=360, dec=-2, dec2=2)
GoldMask_st82 = GoldMask[ pixdec > -3.0 ]
GoldMask_spt = GoldMask[ pixdec < -3.0 ]
#GoldMask_spt = Cuts.SpatialCuts(GoldMask_spt, ra=0, ra2 = 100, dec=-52, dec2 = -48)

pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
SPTMaparea = pixarea * sptnpix
ST82Maparea = pixarea * st82npix

print(SPTMaparea, ST82Maparea)

3726.51398662798 422.0069261713896


In [12]:
from run_systematics import sys_iteration, weightmultiply, fitting_allSP, calling_sysMap

input_path = '/fs/scratch/PCON0008/warner785/bwarner/pca_maps_jointmask_no_stars1623/'
#y3/band_z/
input_keyword = 'pca'
sysMap = io.SearchAndCallFits(path = input_path, keyword = input_keyword)


Calling catalogs >>>
pca17_ALL_maps_4096_ring_jointmask.fits.gz
pca25_ALL_maps_4096_ring_jointmask.fits.gz
pca31_ALL_maps_4096_ring_jointmask.fits.gz
pca48_ALL_maps_4096_ring_jointmask.fits.gz
pca1_ALL_maps_4096_ring_jointmask.fits.gz
pca24_ALL_maps_4096_ring_jointmask.fits.gz
pca18_ALL_maps_4096_ring_jointmask.fits.gz
pca13_ALL_maps_4096_ring_jointmask.fits.gz
pca40_ALL_maps_4096_ring_jointmask.fits.gz
pca11_ALL_maps_4096_ring_jointmask.fits.gz
pca49_ALL_maps_4096_ring_jointmask.fits.gz
pca8_ALL_maps_4096_ring_jointmask.fits.gz
pca27_ALL_maps_4096_ring_jointmask.fits.gz
pca9_ALL_maps_4096_ring_jointmask.fits.gz
pca43_ALL_maps_4096_ring_jointmask.fits.gz
pca21_ALL_maps_4096_ring_jointmask.fits.gz
pca19_ALL_maps_4096_ring_jointmask.fits.gz
pca45_ALL_maps_4096_ring_jointmask.fits.gz
pca6_ALL_maps_4096_ring_jointmask.fits.gz
pca15_ALL_maps_4096_ring_jointmask.fits.gz
pca41_ALL_maps_4096_ring_jointmask.fits.gz
pca29_ALL_maps_4096_ring_jointmask.fits.gz
pca26_ALL_maps_4096_ring_jointmask.f